<a href="https://colab.research.google.com/github/pgordin/OptDisc2024/blob/main/Grafy4_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from random import random
from copy import deepcopy

In [37]:
class Graph:
    def __init__(self, graph=None):
        if graph is None:
            graph = {}
        self.graph = graph

    # dict initializer
    @classmethod
    def from_dict(cls, graph):
        return cls(graph)

    # array initializer
    @classmethod
    def from_array(cls, graph: np.array, nodes: list = None):
        if nodes is None:
            nodes = [*range(1, len(graph) + 1)]
        return cls.from_dict(
            cls._array_to_dict(graph, nodes)
        )

    @staticmethod
    def from_edges(filename: str, directed = 0):
        """
        Generates a graphs from a text file, where each line defines one edge.\n
        Filename is a file path.
        """
        graph = Graph()
        file = open(filename, "r")
        for line in file:
            words = line.strip().split()
            if len(words) == 1:
                graph.add_node(words[0])
            elif len(words) > 1: # more than two words -- we use first two
                if directed:
                    graph.add_arc([words[0], words[1]])
                else:
                    graph.add_edge([words[0], words[1]])
        file.close()
        return graph

    @staticmethod
    def random_graph(nodes_num: int, prob: float):
        """
        Generates a random graph provided a number of nodes and probability of generating an edge.
        """
        rand_graph = Graph()
        for i in range(1, nodes_num + 1):
            rand_graph.add_node(i)
            for j in range(1, i):
                if random() < prob:
                    rand_graph.add_edge([i, j])
        return rand_graph

    @staticmethod
    def cycle(nodes_num: int):
        """
        Generates a cycle provided a number of nodes.
        """
        cycle = Graph()
        nodes = [*range(1, nodes_num + 1)]
        for i in nodes:
            cycle.add_edge([nodes[i-2], nodes[i-1]])
        cycle.graph = dict(sorted(cycle.graph.items()))
        return cycle

    def to_neighbourlist(self, filename: str):
        """
        Saves a graphs to a text file as a neighbour dict.\n
        Filename is a file path.
        """
        file = open(filename, "w")
        file.write(str(self))
        file.close()

    def nodes(self) -> list:
        """
        Returns list of nodes of a graph.
        """
        return [*self.graph.keys()]

    def array(self) -> np.array:
        """
        Returns the graph in array form.
        """
        return self._dict_to_array(self.graph)

    # redefinition of print for objects of class Graph
    def __str__(self):
        res = ""
        for v in self.graph:
            res += f"{v}:"
            for u in self.graph[v]:
                res += f" {u}"
            res += "\n"
        return res

    def add_node(self, node):
        """
        Adds a node to a graph.
        """
        if node not in self.graph:
            self.graph[node] = []

    def del_node(self, node):
        """
        Recursively removes a node from a graph.
        """
        if node in self.graph:
            self.graph.pop(node)
            for key in [*self.graph.keys()]:
                if node in self.graph[key]:
                    self.graph[key].remove(node)

    def add_arc(self, arc: list):
        """
        Adds arc to a graph provided a list of nodes.
        """
        u, v = arc
        self.add_node(u)
        self.add_node(v)
        if v not in self.graph[u]:
            self.graph[u].append(v)

    def add_edge(self, edge: list):
        """
        Adds edge to a graph provided a list of nodes.
        """
        u, v = edge
        if u == v:
            raise ValueError("Pętla!")
        self.add_node(u)
        self.add_node(v)
        if v not in self.graph[u]:
            self.graph[u].append(v)
        if u not in self.graph[v]:
            self.graph[v].append(u)

    def _array_to_dict(arr: np.array, nodes: list) -> dict:
        """
        Converts a graph in array form to a graph in dict form.
        """
        res_dict = {}
        for i, node in enumerate(nodes):
            neighbours = [nodes[j] for j, edge in enumerate(arr[i]) if edge]
            res_dict[node] = neighbours
        return res_dict

    def _dict_to_array(self, _dict: dict) -> np.array:
        """
        Converts a graph in dict form to a graph in array form.
        """
        n = len(_dict)
        nodes = [*_dict.keys()]
        res_arr = np.zeros(shape = (n, n), dtype=int)
        for u,v in [
            (nodes.index(u), nodes.index(v))
            for u, row in _dict.items() for v in row
        ]:
            res_arr[u][v] += 1
        return res_arr

    def Prufer(self):
      """
      Kod Prufera drzewa - zwrócony jako napis
      Wymagane, aby graf był drzewem.
      Zwraca pusty napis dla drzew o mniej niż 3 wierzchołkach.
      """
      tr = deepcopy(self.graph)   # będziemy psuć graf
      code = ""
      for i in range(len(self.graph) - 2):
        for x in sorted(tr):    # po kolei przeglądam nieusunięte wierzchołki
          if len(tr[x]) == 1:   # najmniejszy liść
            break
        v = tr[x][0]  # sąsiad najmniejszego x
        code = code + f"{v} "
        tr[v].remove(x)   # usuwam x z listy sąsiadów v
        tr.pop(x)         # usuwam x z drzewa
      return code.strip()

    @staticmethod
    def tree_from_Prufer(code: str):
        """
        Tworzy drzewo na podstawie kodu Prufera.
        """
        tree = Graph()
        clist = [int(x) for x in code.strip().split()]   # kod zamieniamy na listę liczb
        n = len(clist) + 2    # liczba wierzchołków
        vert = [v for v in range(1, n+1)]  # lista liczb od 1 do n
        for v in vert:
          tree.add_node(v)
        for i in range(n-2):
          for x in vert:
            if not x in clist:    # najmniejszy liść
              break
          v = clist.pop(0)    # usuwam pierwszy element listy (sąsiad x)
          tree.add_edge((x, v))
          vert.remove(x)
        tree.add_edge(vert)
        return tree

    def ConnectedComponents(self):
      """
      Znajduje spójne składowe w grafie nieskierowanym
      Jako wynik zwraca listę zbiorów wierzchołków
      Uwaga: jako pierwszy element listy uzyskamy zbiór wszystkich wierzchołków grafu
      """
      def DFS(u):
        """
        Przeszukiwanie w głąb
        """
        for w in self.graph[u]:
          if not w in VT[0]:  # w jeszcze nie odwiedzony
            VT[0].add(w)      # już odwiedzony
            VT[-1].add(w)     # w ostatniej spójnej składowej
            DFS(w)
      """
      VT - lista zbiorów VT[i] dla i > 0 - lista wierzchołków spójnych składowych
      dla i = 0 - lista wszystkich odwiedzonych wierzchołków
      """
      VT = [set([])]
      for v in self.graph:
        if v not in VT[0]:
          VT[0].add(v)
          VT.append(set([v]))   # zaczątek nowej spójnej składowej
          DFS(v)
      return VT

    def Connected_components_graphs(self):
      """
      Spójne składowe jako grafy
      """
      VT=self.ConnectedComponents()
      noCc=len(VT)-1  #liczba spojnych skladowych
      graphs=[]  #lista do przechowywania grafow reprezentujacych spojne skladowe
      for i in range(noCc):  #iterujemy przez wszystkie spojne skladowe
          graph_i={}    #dla każdej spójnej składowej tworzymy nowy graf graph_i, który jest podgrafem oryginalnego grafu,
          for v in VT[i+1]:     #zawierającym tylko wierzchołki z bieżącej spójnej składowej
              graph_i[v]=self.graph[v].copy()
          graphs.append(Graph.from_dict(graph_i))  #dodajemy stworzone grafy do listy
      return graphs

    def Distance(self, v):
      """
      Znajduje i zwraca jako wektor słownik odległości od wierzchołka v
      do wierzchołków w tej samej spójnej składowej co v
      """
      dist = {v:0} # zalążek słownika odległości
      kolejka = [v]
      while len(kolejka) > 0:
        u = kolejka.pop(0)
        for w in self.graph[u]:
          if not w in dist:
            dist[w] = dist[u] + 1
            kolejka.append(w)
      return dist

# Przykłady wykorzystania

In [38]:
graph = Graph.random_graph(10, 1/5)
print(graph)
print("---------------------------")
print(graph.ConnectedComponents())


1: 2 3
2: 1 6
3: 1 4 6 8
4: 3 7 8
5: 8
6: 2 3 7 8
7: 4 6
8: 3 4 5 6
9:
10:

---------------------------
[{1, 2, 3, 4, 5, 6, 7, 8, 9, 10}, {1, 2, 3, 4, 5, 6, 7, 8}, {9}, {10}]


In [39]:
odl = graph.Distance(1)
print(odl)

{1: 0, 2: 1, 3: 1, 6: 2, 4: 2, 8: 2, 7: 3, 5: 3}


#Small world phenomenon - eksperyment Millgrama (1967)

Użyjemy grafu $G(n,p)$ (dokładnie $G(2000, 1/300)$) jako ilustrację zjawiska

In [44]:
import sys

n = 2000
p = 1/300
sys.setrecursionlimit(n+5)

rgraph = Graph.random_graph(n, p)
lista = rgraph.Connected_components_graphs()
graph = lista[0]
print(len(graph.graph))


1998


In [45]:
md = {}
ecc = {}
for v in graph.graph:
  dist = graph.Distance(v)
  ecc[v] = max(dist.values())
  md[v] = sum(dist.values())/len(dist.values())
print("Promień:", min(ecc.values()), " Średnica:", max(ecc.values()), " Średnio: ", sum(md.values())/len(md.values()))

Promień: 5  Średnica: 8  Średnio:  4.2265027790553304


In [25]:
graph.add_edge((11, 12))
print(graph.ConnectedComponents())


[{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12}, {1, 2, 3, 4, 6, 7, 8, 9, 10}, {5}, {11, 12}]


In [26]:
lista = graph.Connected_components_graphs()

In [27]:
for graph1 in lista:
  print(graph1)
  print("-----------------")

1: 2 8
2: 1 4 6
3: 9
4: 2
6: 2 9
7: 8
8: 1 7 10
9: 3 6 10
10: 8 9

-----------------
5:

-----------------
11: 12
12: 11

-----------------


In [28]:
graph.add_edge((6, 11))

In [29]:
for graph1 in lista:
  print(graph1)
  print("-----------------")

1: 2 8
2: 1 4 6
3: 9
4: 2
6: 2 9
7: 8
8: 1 7 10
9: 3 6 10
10: 8 9

-----------------
5:

-----------------
11: 12
12: 11

-----------------


In [ ]:
%%writefile lista.txt
A B
B C
B D
D C
E
F

Writing lista.txt


In [ ]:
file_graph = Graph.from_edges("./lista.txt")
print(file_graph)

A: B
B: A C D
C: B D
D: B C
E:
F:



In [ ]:
file_graph.add_edge(["E", "F"])
file_graph.to_neighbourlist("lista1.txt")

In [ ]:
%cat lista1.txt

A: B
B: A C D
C: B D
D: B C
E: F
F: E


In [34]:
!wget https://raw.githubusercontent.com/pgordin/OptDisc2024/main/ubranie.txt

--2024-04-22 11:00:59--  https://raw.githubusercontent.com/pgordin/OptDisc2024/main/ubranie.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 186 [text/plain]
Saving to: ‘ubranie.txt’

ubranie.txt         100%[===================>]     186  --.-KB/s    in 0s      

2024-04-22 11:00:59 (2.10 MB/s) - ‘ubranie.txt’ saved [186/186]



In [35]:
ubranie_graph = Graph.from_edges("ubranie.txt", directed = 1)
print(ubranie_graph)

slipki: kalesony
kalesony: spodnie
spodnie: buty szelki
buty:
szelki: marynarka
skarpety: buty
koszula: szelki marynarka krawat
marynarka: plaszcz
krawat: marynarka
plaszcz:

